# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "random_cities.csv"
df = pd.read_csv(csv_path)

df.head()

,City,Country,Cloudiness,date,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed
0,Faya,SA,0,1569364574,60,18.39,42.45,69.80,2.01
1,Yellowknife,CA,75,1569364574,61,62.45,-114.38,46.40,14.99
2,Hilo,US,75,1569364575,58,19.71,-155.08,87.80,9.17
3,Ushuaia,AR,40,1569364556,75,-54.81,-68.31,44.60,28.86
4,Chuy,UY,0,1569364384,87,-33.69,-53.46,54.98,7.81


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Defining variables that gmaps will use
locations = df[["Latitude", "Longitude"]].astype(float)
humidity = df["Humidity"].astype(float)

# Centering the zoom of the maps
fig = gmaps.figure(center=(0, 0), zoom_level=3)

# Adding heat layer with weights based on humidity
heat_layer = gmaps.heatmap_layer(locations, humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# I'm a Seattleite, so I'll say my ideal weather conditions are:
#     * Between 60 and 70 degrees F
#     * Cloudiness at least 75%
#     * Windspeed less than 10 MPH
#     * Humidity between 70 and 80%

best_df = df.loc[(df['Maximum Temp'] >= 60) & (df['Maximum Temp'] < 70)
                 & (df['Cloudiness'] >= 75) & (df['Wind Speed'] <= 10)
                & (df['Humidity'] >= 70) & (df['Humidity'] < 80)]

# Drop rows with null values
best_df = best_df.dropna()

# View dataframe sorted by city name
best_df.sort_values(by="City")

,City,Country,Cloudiness,date,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed
407,Beloha,MG,92,1569364723,78,-25.17,45.06,68.30,7.20
452,Bugene,TZ,85,1569364738,72,-1.59,31.14,67.22,2.91
185,Codrington,AU,75,1569364643,72,-28.95,153.24,68.00,4.21
103,Katsuura,JP,75,1569364611,73,33.93,134.50,69.80,9.17
296,Richmond,CA,75,1569364408,77,49.16,-123.14,66.99,9.17
126,Saint-Augustin,CA,90,1569364620,77,45.63,-73.98,60.80,4.70
117,Santa Teresa,BR,90,1569364617,77,-19.94,-40.60,68.00,4.25
10,Xai-Xai,MZ,79,1569364577,74,-25.04,33.64,64.70,9.62


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Store into variable named hotel_df.
hotel_df = best_df

In [7]:
# Add a "Hotel Name" column to the DataFrame. 
hotel_df["Hotel Name"] = ""

In [8]:
# Defining parameters for Google Place API search.
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# Loop to identify nearby hotels via Google Place.
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]

    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    

    name_address = name_address.json()

    # Check for errors
    try:
        best_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
    # Replace blanks with NaN, so that they can be dropped later
        best_df.loc[index, "Hotel Name"] = np.nan
        print(f"Could not find nearby hotel in {row['City']}.")

Could not find nearby hotel in Katsuura.
Could not find nearby hotel in Saint-Augustin.
Could not find nearby hotel in Beloha.


In [9]:
# Dropping rows where a hotel could not be found.
hotel_df = hotel_df.dropna()

# Check updated table
hotel_df.sort_values(by="City")

,City,Country,Cloudiness,date,Humidity,Latitude,Longitude,Maximum Temp,Wind Speed,Hotel Name
452,Bugene,TZ,85,1569364738,72,-1.59,31.14,67.22,2.91,Sunrise Annex Hotel
185,Codrington,AU,75,1569364643,72,-28.95,153.24,68.00,4.21,Coraki Hotel
296,Richmond,CA,75,1569364408,77,49.16,-123.14,66.99,9.17,Pacific Gateway Hotel at Vancouver Airport
117,Santa Teresa,BR,90,1569364617,77,-19.94,-40.60,68.00,4.25,Hotel Solar dos Colibris
10,Xai-Xai,MZ,79,1569364577,74,-25.04,33.64,64.70,9.62,Platinum Hotel


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in best_df.iterrows()]
marker_locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Establishing markers for hotels
markers = gmaps.marker_layer(marker_locations)

# Adding heat layer with weights based on humidity
heat_layer = gmaps.heatmap_layer(locations, humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

# Add marker layer ontop of heat map
fig.add_layer(markers)

# Centering the zoom of the maps
fig

Figure(layout=FigureLayout(height='420px'))